# Talent Intelligence System ( Data Exploration)

This notebook serves as the primary analysis environment for **Step 1: Discover the Pattern of Success**.

**Objective:** The primary goal of this notebook is to perform a deep exploratory data analysis (EDA) to uncover the key attributes, competencies, and behavioral patterns that differentiate high-performing employees (those with a `rating = 5`) from their peers.

**Process:**
The analysis will follow these key steps:
1.  **Setup:** Import all necessary libraries and establish a connection to the Supabase database.
2.  **Data Loading:** Load all raw tables from the database into Pandas DataFrames.
3.  **Data Cleaning & Preprocessing:** Handle missing values, standardize inconsistent text data (like `mbti`), and create a master analysis DataFrame.
4.  **Exploratory Data Analysis (EDA):** Compare the "High Performer" group against all others across several key dimensions:
    * Psychometric & Cognitive (`iq`, `pauli`, `disc`, etc.)
    * Competencies (`competencies_yearly`)
    * Behavioral Strengths (`strengths`)
    * Contextual Factors (`grade_id`, `years_of_service_months`)
5.  **Synthesis:** Use visualizations and statistical summaries to identify the most significant factors.

**End Goal:** The insights gathered here will be synthesized into a **"Final Success Formula"**. This formula will provide the logical foundation for the SQL matching algorithm to be built in Step 2.

## Setup Preparation

In [1]:
# Install neccessary ilbraries
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load the secret credentials from  .env file

load_dotenv()

DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')

print("Environment variables loaded.")


Environment variables loaded.


In [3]:
# Create the connection 'engine' to  Supabase database

connection_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_string)

print("Database connection engine created successfully.")

Database connection engine created successfully.


In [4]:
# Overview of certain tables for intial checking
df_employees = pd.read_sql(
  
  """
  SELECT 
  * 
  FROM employees
  """

  , engine)
print("\n--- Employee Data Info ---")
display(df_employees.info(), df_employees.head())


--- Employee Data Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2010 entries, 0 to 2009
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   employee_id              2010 non-null   object
 1   fullname                 2010 non-null   object
 2   nip                      2010 non-null   object
 3   company_id               2010 non-null   int64 
 4   area_id                  2010 non-null   int64 
 5   position_id              2010 non-null   int64 
 6   department_id            2010 non-null   int64 
 7   division_id              2010 non-null   int64 
 8   directorate_id           2010 non-null   int64 
 9   grade_id                 2010 non-null   int64 
 10  education_id             2010 non-null   int64 
 11  major_id                 2010 non-null   int64 
 12  years_of_service_months  2010 non-null   int64 
dtypes: int64(10), object(3)
memory usage: 204.3+ KB


None

,employee_id,fullname,nip,company_id,area_id,position_id,department_id,division_id,directorate_id,grade_id,education_id,major_id,years_of_service_months
0,EMP100000,Rendra Pratama,806137,1,4,1,6,3,3,2,3,1,64
1,EMP100001,Wulan Setiawan,476388,4,4,4,5,5,3,1,1,2,16
2,EMP100002,Julia Jatmiko Situmorang,941921,1,3,5,1,1,3,1,3,1,58
3,EMP100003,Oka Halim,751615,1,3,4,2,1,1,2,2,5,15
4,EMP100004,Dwi Pratama,443809,3,4,6,5,3,3,1,1,5,34


## Data Load and Preparation

To optimize performance and avoid excessive load on the database (as per quota concerns), we will use a hybrid loading strategy:

1.  **Main Query (`df_main`):** A single SQL query will join all **one-to-one** tables (`employees`, `profiles_psych`) and the latest performance record (`performance_yearly` filtered for 2025). This creates our primary, unique-per-employee analysis table.
2.  **Separate Queries:** All **one-to-many** tables (`competencies_yearly`, `strengths`) will be loaded separately. They will be merged in Python *only when needed* for a specific analysis to prevent row explosion.

### Load for One to One Data

In [ ]:
#  Load Main One to One Data
df_main= pd.read_sql(
  
  """
  SELECT
    e.*,
    pp.pauli,
    pp.faxtor,
    pp.disc,
    pp.disc_word,
    pp.mbti,
    pp.iq,
    pp.gtq,
    pp.tiki,
    py.rating,
    py.year as current_year
  FROM employees as e
  LEFT JOIN  profiles_psych as pp
  ON e.employee_id = pp.employee_id 
  LEFT JOIN   
  (SELECT * 
  FROM performance_yearly 
  WHERE year = 2025) 
  as py
  ON e.employee_id = py.employee_id

  """
  , engine)
print("\n--- Main Data Info ---")
display(df_main.info(), df_main.head())

print(f"Main analysis table loaded successfully with {len(df_main)} unique employees.")


--- Main Data Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2010 entries, 0 to 2009
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   employee_id              2010 non-null   object 
 1   fullname                 2010 non-null   object 
 2   nip                      2010 non-null   object 
 3   company_id               2010 non-null   int64  
 4   area_id                  2010 non-null   int64  
 5   position_id              2010 non-null   int64  
 6   department_id            2010 non-null   int64  
 7   division_id              2010 non-null   int64  
 8   directorate_id           2010 non-null   int64  
 9   grade_id                 2010 non-null   int64  
 10  education_id             2010 non-null   int64  
 11  major_id                 2010 non-null   int64  
 12  years_of_service_months  2010 non-null   int64  
 13  pauli                    2010 non-null   int64  
 14  

None

,employee_id,fullname,nip,company_id,area_id,position_id,department_id,division_id,directorate_id,grade_id,...,pauli,faxtor,disc,disc_word,mbti,iq,gtq,tiki,rating,current_year
0,EMP100000,Rendra Pratama,806137,1,4,1,6,3,3,2,...,86,75,SI,Steadiness-Influencer,None,94.0,33.0,2,3.0,2025
1,EMP100001,Wulan Setiawan,476388,4,4,4,5,5,3,1,...,48,52,DS,Dominant-Steadiness,INTP,94.0,17.0,3,3.0,2025
2,EMP100002,Julia Jatmiko Situmorang,941921,1,3,5,1,1,3,1,...,66,38,DC,Dominant-Conscientious,None,109.0,20.0,3,4.0,2025
3,EMP100003,Oka Halim,751615,1,3,4,2,1,1,2,...,39,63,SI,Steadiness-Influencer,ENTJ,85.0,39.0,3,3.0,2025
4,EMP100004,Dwi Pratama,443809,3,4,6,5,3,3,1,...,75,100,None,Steadiness-Conscientious,INTJ,134.0,21.0,9,4.0,2025


Main analysis table loaded successfully with 2010 unique employees.


### Load One to Many Data Seperately

In [17]:
# Competencies table
df_competencies= pd.read_sql(
  
  """
  SELECT
    cy.employee_id,
    cy.pillar_code,
    dc.pillar_label,
    cy.score,
    cy.year
  FROM competencies_yearly as cy
  LEFT JOIN dim_competency_pillars as dc
  ON cy.pillar_code = dc.pillar_code
  WHERE 
  cy.year = 2025
  """
  , engine)
print("\n--- Competencies Info ---")
display(df_competencies.info(), df_competencies.head())
print(f"Competency data loaded with {len(df_competencies)} rows.")


--- Competencies Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20100 entries, 0 to 20099
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   employee_id   20100 non-null  object 
 1   pillar_code   20100 non-null  object 
 2   pillar_label  20100 non-null  object 
 3   score         18495 non-null  float64
 4   year          20100 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 785.3+ KB


None

,employee_id,pillar_code,pillar_label,score,year
0,EMP100000,GDR,Growth Drive & Resilience,3.0,2025
1,EMP100001,GDR,Growth Drive & Resilience,2.0,2025
2,EMP100002,GDR,Growth Drive & Resilience,3.0,2025
3,EMP100003,GDR,Growth Drive & Resilience,4.0,2025
4,EMP100004,GDR,Growth Drive & Resilience,4.0,2025


Competency data loaded with 20100 rows.


In [18]:
# Strengths table
df_strengths= pd.read_sql(
  
  """
  SELECT
    employee_id,
    rank,
    theme
  FROM strengths
  ORDER BY 1 ASC, 2 ASC

  """
  , engine)
print("\n--- Strengths Info ---")
display(df_strengths.info(), df_strengths.head())
print(f"Strengths data loaded with {len(df_strengths)} rows.")


--- Strengths Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28140 entries, 0 to 28139
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   employee_id  28140 non-null  object
 1   rank         28140 non-null  int64 
 2   theme        28140 non-null  object
dtypes: int64(1), object(2)
memory usage: 659.7+ KB


None

,employee_id,rank,theme
0,DUP1942,1,Learner
1,DUP1942,2,Maximizer
2,DUP1942,3,Ideation
3,DUP1942,4,Harmony
4,DUP1942,5,Restorative


Strengths data loaded with 28140 rows.


In [20]:
# Papi_scores table
df_papi_scores= pd.read_sql(
  
  """
  SELECT
    employee_id,
    scale_code,
    score
  FROM papi_scores
  ORDER BY 1 ASC, 2 ASC
  """
  , engine)
print("\n--- PAPI Scores Info ---")
display(df_papi_scores.info(), df_papi_scores.head())
print(f"PAPI Scores data loaded with {len(df_papi_scores)} rows.")


--- PAPI Scores Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40200 entries, 0 to 40199
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   employee_id  40200 non-null  object 
 1   scale_code   40200 non-null  object 
 2   score        36997 non-null  float64
dtypes: float64(1), object(2)
memory usage: 942.3+ KB


None

,employee_id,scale_code,score
0,DUP1942,Papi_A,6.0
1,DUP1942,Papi_B,NaN
2,DUP1942,Papi_C,1.0
3,DUP1942,Papi_D,4.0
4,DUP1942,Papi_E,3.0


PAPI Scores data loaded with 40200 rows.
